In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../..')
from helper.lazy_regression import TooLazyForRegression
from helper.feature_extract import extract_highest_amplitude_features_with_mp
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from scipy.stats import boxcox
from scipy.special import inv_boxcox
from scipy.stats import yeojohnson
import seaborn as sns
import matplotlib.pyplot as plt

### featured data:

In [267]:
df_spg = pd.read_table('../../data/data_spg.txt', sep=' ')
df_spg = extract_highest_amplitude_features_with_mp(df=df_spg, create_one_sensor_feature=True, n_processes=4, keep_columns=False,verbose=False)
df_spg = df_spg[df_spg['start_time'].str.contains('2021')]
df_spg = df_spg.drop(["start_time","packnr","velocity"], axis=1)

INFO || Extracting Max Features for types: ['G01', 'G02']


In [268]:
X, y = df_spg.drop(['size_mm'], axis=1), df_spg['size_mm']
y =y.astype(np.str)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state = 102)

In [269]:
X, y = df_spg.drop(['size_mm'], axis=1), df_spg['size_mm']
scores = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1)
    model= xgb.XGBRegressor()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_pred,y_test)
    scores.append(r2)

In [270]:
np.mean(scores)

0.7008319514443808

## raw data:

In [2]:
data_raw_spg = "../../data/raw_packets_obernach_GP_0.02_2510.vlnd"
df_spg = pd.read_table('../../data/data_spg.txt', sep=' ')

In [3]:
def df_coverter(data):
    with open(data) as f:
        lines = f.readlines()
    raw = pd.DataFrame(lines) 
    df = raw[0].str.split(' ', expand=True)
    return df

In [7]:
df_raw_spg = df_coverter(data_raw_spg)

In [8]:
df_spg["start_time"] = df_spg["start_time"] + " " + df_spg["packnr"].astype(str)

In [9]:
df_raw_spg["start_time"] = df_raw_spg[0] + " " + df_raw_spg[1] + " " + df_raw_spg[2]

In [14]:
new_raw_data = pd.merge(df_spg,df_raw_spg,on='start_time',how='left')

In [247]:
new_raw_data = new_raw_data[new_raw_data['start_time'].str.contains('2021')]

In [248]:
new_raw_data = new_raw_data.drop([["start_time","packnr","velocity"]], axis=1)

In [249]:
new_raw_data = new_raw_data.drop([0,1,2,3], axis=1)

In [250]:
new_raw_data.iloc[:, :] = new_raw_data.iloc[:, :].astype("float")

In [264]:
X, y = new_raw_data.drop(['size_mm'], axis=1), new_raw_data['size_mm']
scores = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1)
    model= xgb.XGBRegressor()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_pred,y_test)
    scores.append(r2)

In [265]:
np.mean(scores)

0.7539972002623017

### Fazit

Das vorbereitete Datensatz inklusiv mit den Rohdaten schneidet sehr gut ab und ist besser.